### Motivation
* This notebook is to create a trusted San Diego County and Imperial County block groups on a feature layer
We will try to see if we can use the existing feature layer created from Esri :  
Feature Layer: https://services.arcgis.com/P3ePLMYs2RVChkJx/arcgis/rest/services/USA_Census_BlockGroups/FeatureServer  
Feature Layer Details: https://esri.maps.arcgis.com/home/item.html?id=2f5e592494d243b0aa5c253e75e792a4
* Using this layer, we would pick San Diego County and Imperial County FIPS data  
* Enrich the Dataframe with consumer spending and other variables

In [2]:
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection, FeatureLayer
from arcgis.geoenrichment import enrich, Country
from feature_layer_utils import get_enrichment_variables
import itertools
import sys
sys.path.append('../')
from utils import get_config

In [3]:
app_id = get_config("arcgis","clientid")
gis = GIS("https://ucsdonline.maps.arcgis.com/home", client_id=app_id)

Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://ucsdonline.maps.arcgis.com/sharing/rest/oauth2/authorize?response_type=code&client_id=Elm5V3upnnV17Q3r&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=FZOZm78VxzOUhyfdo0OE9BwnfSbBHL&allow_verification=false


Enter code obtained on signing in using SAML:  ········


/Users/amolkale/Documents/DSE/DSE260-Capstone-Project/capstone/venv3.9/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ucsdonline.maps.arcgis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


#### Selecting only San Diego and Imperial County block groups from US block groups

In [4]:
fls = "https://services.arcgis.com/P3ePLMYs2RVChkJx/arcgis/rest/services/USA_Census_BlockGroups/FeatureServer"

In [5]:
flc = FeatureLayerCollection(fls)

In [6]:
fl_url = flc.layers[0].url

In [7]:
us_block_grp_fl = FeatureLayer(fl_url)

Selecting FIPS=`073` which is for San Diego County as our data and Imperial county FIPS=`025`

In [8]:
san_diego_county_fip = '073'
imperial_county_fip = '025'

In [11]:
san_diego_imperial_county_block_grp_fs = us_block_grp_fl.query(
    where=f"STATE_FIPS='06' AND (COUNTY_FIPS='{san_diego_county_fip}' OR COUNTY_FIPS='{imperial_county_fip}')"
)

In [12]:
san_diego_imperial_county_block_grp_fs_sdf = san_diego_imperial_county_block_grp_fs.sdf
print(f"Shape: {san_diego_imperial_county_block_grp_fs_sdf.shape}")
san_diego_imperial_county_block_grp_fs_sdf.head(4)

Shape: (2171, 14)


,OBJECTID,STATE_ABBR,STATE_FIPS,COUNTY_FIPS,STCOFIPS,TRACT_FIPS,BLOCKGROUP_FIPS,FIPS,POPULATION,POP_SQMI,SQMI,Shape__Area,Shape__Length,SHAPE
0,14491,CA,06,025,06025,010101,1,060250101011,3499,23.8,146.77,0.036755,1.165018,"{""rings"": [[[-115.452063638766, 33.25680691500..."
1,14492,CA,06,025,06025,010102,1,060250101021,1128,7.0,161.66,0.040486,0.957515,"{""rings"": [[[-115.519519650705, 33.24369790988..."
2,14493,CA,06,025,06025,010102,2,060250101022,1228,1980.6,0.62,0.000155,0.064301,"{""rings"": [[[-115.501922638842, 33.13298389730..."
3,14494,CA,06,025,06025,010102,3,060250101023,1534,3652.4,0.42,0.000104,0.041999,"{""rings"": [[[-115.519082641878, 33.12576989492..."


For enrichment, we can provide the feature set or the spatially enabled df.
Let us just pass down the sdf since we have visualized it already.

In [14]:
san_diego_imperial_block_groups_fl = san_diego_imperial_county_block_grp_fs_sdf.spatial.to_featurelayer(title='San Diego and Imperial County Block Groups',
                                                                                                     gis=gis,
                                                                                                     folder='nourish_gis',
                                                                                                     tags=['SanDiego','Imperial','BlockGroups','County'],
                                                                                                     service_name='San Diego and Imperial County Block Groups'
                                                                                                    )
san_diego_imperial_block_groups_fl

<Item title:"San Diego and Imperial County Block Groups" type:Feature Layer Collection owner:akale_UCSDOnline>